# 文本分析


何璇 15300681068

## 问题阐述
微信是人们社交网络的重要组成部分，微信公众号也渐渐成为当代各类型媒体的传播平台：其中既有个人、小团队经营的自媒体，又有各类企业、乃至各级政府的官方媒体。微信公众号中的文章内容与主题丰富而多变，将文章进行分门别类，一方面便于日后的深入分析，另一方面也体现出该公众号发表文章的性质与特征。但是，通常作者撰写文章时不会自主将其归为某一特定主题，所以当我们获取微信文章进行分析时，面临的是大量无标签的样本。对这类样本有两种解决方法：一是对所有训练样本进行人工分类，但这样将耗费大量人力，且难免产生一定的错误与遗漏；二是采用非监督学习的方法，让算法对这些文章自动分类，从而免去了人工分类的过程。

我选用自己一直在关注的果壳公众号文章作为样本。为了测试算法的可靠性，首先我将算法用于选取的四十余篇文章，通过对小样本分类结果的分析，我们可以选择进一步优化算法，或者将其运用于人工难以标记的大型样本。

## 流程

<img src="流程图1.png" width="60%">

以上流程主要分为三块：数据导入、数据预处理与数据分析。


1. 数据导入

   对于小样本，我们已经拥有每篇文章的txt文档，存放于selected_articles的子目录中，文档名称即为文章标题。可直接采用遍历将文档导入。对于所有样本，我们已知每篇文章的url地址，所有的地址连同标题等其他信息均存放在csv格式的文件中，我们将所有的html文档中的正文内容提取并以txt文档形式分篇保存，如此便可同之前的方法导入数据。
   
   
2. 数据预处理

   由于数据为文本格式，我们需要对其进行分词、去停用词操作。然后选取能够描述所有文章的特征。这里我们采用向量空间模型对文章进行描述。由于文章数量越大，词汇量随之增大，这将导致特征空间的维度过大，数据的分布过于稀疏，进而影响后续聚类算法的效果。我们只选取一定数量量的词汇（关键词）作为特征。最后，我们对权重矩阵进一步预处理，如：标准化。
   
   
3. 数据分析

   我们采用无监督学习中的聚类算法将文章分为特定数量的类，并对分类结果进行分析和讨论。

## 实现技术

1. 编程环境：Python 3.6(Anaconda)
2. 外部库：jieba, bs4, numpy, pandas, sklearn.

In [1]:
#调用所需模块
import jieba
from jieba import posseg
from itertools import groupby
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import os
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# 导入停用词表，并添加与公众号本身相关的词
stopwords_list = [line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8').readlines()]
stopwords_list.append('果壳')
stopwords_list.append('果壳网')
stopwords_list.append('图')
stopwords_list.append('图片')
stopwords_list.append('科普')

### 数据导入

首先我们导入小样本中的文本，将文章标题、正文内容按顺序存放于numpy数组中。在导入的同时，我们同时对正文内容进行分词与去停用词，并存放于新的数组中，这是为了方便后续取用，避免对文本重复进行分词操作。方便起见，对于分词后的结果，我们只选择中文词汇，而忽略其他，如：英文词汇、标点。

In [3]:
# 定义文档存放路径
article_path = r'F:\\selected articles\\'

In [14]:
# 由于还没有进行函数的定义，我们先不运行这一模块，而在函数定义之后运行
# 存放文章标题
article_names = []
# 存放原始文本内容
article_texts = []
# 存放分词后的文本内容
seg_articles = []
# 存放分词、去停用词后的文本内容
seg_articles_delstp = []

# 一一读取目录中的文档
for fname in os.listdir(article_path):
    article = open(os.path.join(article_path, fname),'r',encoding='utf-8').read()
    
    article_name = fname.replace('.txt', '')
    article_names.append(article_name)
    
    article_text = article.replace('\n', '')
    article_texts.append(article_text)
    # 分词
    article_seg_list = jieba.lcut(article_text)
    
    seg_text = ''
    seg_text_delstp = ''
    
    for word in article_seg_list:
        if is_chinese(word):
            seg_text += word + ' '
            if word not in stopwords_list:
                seg_text_delstp += word + ' '
                
    seg_articles.append(seg_text)
    seg_articles_delstp.append(seg_text_delstp)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.951 seconds.
Prefix dict has been built succesfully.


### 定义函数

下面我们将定义数据处理、分析过程中的函数。

In [4]:
# 筛选中文字符
def is_chinese(uchar):

        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True

        else:
                return False

之前的数据导入过程中，我们已经顺带将文本分词、停用词步骤完成。因为正文长度较长（通常几千字），我们针对正文内容进行了预先的处理，存储了预处理后的文本。对于标题，由于通常只有十几个字，所以选择在需要的时候再进行现成的处理操作，这样既不浪费空间，对算法运行时间也不会造成明显的影响。

定义对标题分词、去停用词的函数。

In [5]:
# 从文本中获取分词、去停用词后的中文词汇，以列表格式返回
def get_words_list_del_stpwds(text):
    # 分词
    seg_list = jieba.lcut(text)
    
    seg_list_chin = []
    for i in range(len(seg_list)):
        word = seg_list[i]
        # 选取不在停用词表的中文词汇
        if is_chinese(word) and word not in stopwords_list:
                seg_list_chin.append(word)
    
    return seg_list_chin

由于我们采用向量空间模型，特征由词汇在文章中的权重表示。首先，计算词汇在文章中出现的词频。

In [6]:
# 获取词汇列表中每个词的频次，以词典形式返回
def get_words_dict(words_list):
    # 排序
    words_list.sort()
    # 计算每个词汇的频次
    words_dict = {}
    for key, group in groupby(words_list):
        words_dict[key] = len(list(group))
        
    return words_dict

如果将所有文章中出现的词汇全部做为特征，则特征数量将远远大于文档数量，样本在高维空间分布极其稀疏，将造成“维度灾难”，不利于后续的分析。所以对特征空间进行降维尤其重要，要尽可能选取数量少但是能很好区分文章主题的关键词作为特征。特征选择有有监督与无监督两类，但是对于无标签样本集，无法使用有监督的方法，只能退而求其次，采用无监督的选择方式，这种方式的缺点主要在于：我们不知道选取的特征能否很好地区别文本，没有被选择的特征中也可能包含重要的信息。

对于小样本，因为文章数量少，词汇也较少，我们仍然有可能不进行任何降维操作，直接对不同文档聚类，然后与降维后的结果比较，有助于分析对比降维操作对数据带来的影响。

接下来，我们讨论如何选取关键词。

首先，我们并非将每个文档的所有词汇收入特征集，而是假设：对于一篇文章，最能表现其主题含义的是出现频率较高的一些词（此时已经对文章做了去停用词操作），所以只要选取前n大的词汇作为关键词即可。

这种假设具有一定的依据：与主题相关的词汇总在文章中反复出现，所以高频词能够反映文章的内容。但是并非完美，因为即便使用停用词表过滤了无用、常用词，依然可能存在一些与主题无关的高频词汇。而且对于无监督方式，我们还面临一个问题，应该选取前多少个词汇作为关键词？缺乏有效的反馈，对n的选取只能来源于对维度的要求，如果要求维度尽量小，则n也应变小，此时就需要承担信息损失的风险。

为了观察降维操作是否带来明显的信息损失，进而影响聚类的结果，我们充分利用小样本，用原始数据聚类的结果与降维后的结果进行对照。

接下来定义函数实现对前n个高频词的提取。

In [7]:
# 对一篇文章，输入词频词典，获得最高频的n个词汇
def get_most_frequent_words(words_dict, n):
    
    frequent_words = sorted(words_dict, key=words_dict.get, reverse=True)[:n]
            
    return frequent_words

In [8]:
# 将所有文章正文出现频次最高的n个词作为关键词，返回集合形式
def get_key_words_from_content(seg_articles_delstp, n):
    # 定义集合存放关键词
    n_articles = len(seg_articles_delstp)
    content_key_words = set()
    for i in range(n_articles):
        # 将文本转换为列表，一个元素为一个词汇
        seg_list = seg_articles_delstp[i].split()
            
        words_dict = get_words_dict(seg_list)
        frequent_words = get_most_frequent_words(words_dict, n)
        
        content_key_words.update(frequent_words)
        
    return content_key_words

In [9]:
# 由于前面只获取了正文中的关键词，需要继续考虑标题中的词汇
def get_keywords_from_title(article_names):
    n_articles = len(article_names)
    # 定义集合存放标题关键词
    title_key_words = set()
    for i in range(n_articles):
        title = article_names[i]
        # 提取标题中不属于停用词的中文词汇
        # 对于标题，我们不考虑频率，而是将出现的非停用词一律作为关键词
        seg_title = get_words_list_del_stpwds(title)
        title_key_words.update(seg_title)
        
    return title_key_words

In [10]:
# 将正文中和标题中获得的关键词合并，返回列表格式
def get_keywords(seg_articles_delstp, article_names, n):
    content_key_words = get_key_words_from_content(seg_articles_delstp, n)
    title_key_words = get_keywords_from_title(article_names)
    key_words = content_key_words
    key_words.update(title_key_words)
    return list(key_words)

选取了一定量的关键词后，我们开始计算TF-IDF值。

需要申明的是，此时的降维包括特征选取还没有结束，对于DF值有一定的范围要求：关键词的文档频率不应太小——没有代表性，也不应太大——没有区分度。

Sklearn库中，带有TF-IDF函数，但这里我们不采用这个函数，而是自行定义，这是为了对算法进行控制和改进。

比如，原始的TF-IDF计算不考虑标题和文本的差别，事实上，对于微信文章，标题的内容尤其重要，为了吸引兴趣、使读者对这篇文章拥有大致概念，标题通常明示或暗示文章的主要内容、主题思想，虽然我们的算法不解决语义层面的问题，与主题相关的关键词往往会出现在标题中。标题通常只有十几字，这就要求我们重视标题中出现的每个词汇，对其赋权。标题的权重应当大于正文，但这也造成一些与主题无关，或者单纯承担吸引读者功能的词汇被赋予较高的权重。所以我们应当谨慎选择权重，避免过大和过小。

另外我们之前对DF值的要求也可直接用自定义的算法实现。即去除DF值不符合要求的相应词汇。

In [11]:
# 计算用文档长度进行归一化的词频
def cal_word_freq(seg_articles, article_names, key_words, title_weight):
    
    n_articles = len(seg_articles)
    # 创建初始值均为零的矩阵存放词汇频率，其中行代表文章，列代表词汇
    frequency = np.zeros([n_articles, len(key_words)])
    # 创建数组存放文章的所有中文词汇数量
    article_length = np.zeros(n_articles)
    
    for i in range(n_articles):
        seg_list = seg_articles[i].split()
        # 计算文章所有中文词汇数量
        article_length[i] = len(seg_list)
        # 计算每个词汇的频次
        words_dict = get_words_dict(seg_list)
        # 对每个关键词，计算该文章中出现的归一化频率
        for j in range(len(key_words)):
            key_word = key_words[j]
            if key_word in words_dict:
                frequency[i, j] = words_dict[key_word] / article_length[i]
                
    # 加入标题的加权权重
    for i in range(n_articles):
        title = article_names[i]
        seg_title = get_words_list_del_stpwds(title)
        for word in seg_title:
            j = key_words.index(word)
            frequency[i, j] += title_weight
                
    return frequency, article_length

In [12]:
# 计算逆文档频率，同时返回文档频率不符合要求的词汇
def cal_idf(tf_matrix, min_df, max_df):
    # 计算每个关键词出现的文章数量
    df = np.count_nonzero(tf_matrix, axis=0)
    # 找出DF值不满足范围要求的关键词
    bad_words = np.where((df < min_df) | (df > max_df))
    # 计算逆文档频率
    d = tf_matrix.shape[0]
    idf = np.log(d / df)
    # 如果逆文档频率为零，则赋一个很小的值
    for i in range(len(idf)):
        if idf[i] == 0:
            idf[i] = 0.001
    return idf, bad_words

In [13]:
# 计算TF-IDF值，去掉DF值不满足要求的词
def cal_tf_idf(seg_articles, article_names, key_words, title_weight, min_df, max_df):
    
    tf_matrix, article_length = cal_word_freq(seg_articles, article_names, key_words, title_weight)
    idf, bad_words = cal_idf(tf_matrix, min_df, max_df)
    
    tf_matrix = np.delete(tf_matrix, bad_words, axis=1)
    key_words_upd = np.delete(key_words, bad_words)
    idf = np.delete(idf, bad_words)
    
    idf = idf.reshape(1, -1)
    return tf_matrix * idf, key_words_upd

### 数据预处理

定义函数、导入数据后，我们开始预处理流程。

下面展示数据内容。

In [15]:
n_articles = len(article_names)
print('n_articles: ', n_articles, '\n')
print('article_names: \n')
for name in article_names:
    print(name, '\n')
print('article_texts: ', article_texts[0][:200], '......', '\n')
print('seg_articles: ', seg_articles[0][:200], '......', '\n')
print('seg_articles_delstp: ', seg_articles_delstp[0][:200], '......', '\n')

n_articles:  48 

article_names: 

7位科学家评基因编辑婴儿：不是对新知的探索，也没有医疗意义 

“M型病毒”这个名字哪儿来的？病毒命名有什么规律？ 

“基因编辑婴儿”出现后，设计一个“完美婴儿”离我们还有多远？ 热点 

“艾滋病免疫婴儿”诞生：基因编辑技术，为何成为争议的漩涡？丨热点 

一个灵敏的屁股有什么用 

世界艾滋病日：阻断HIV感染的黄金24小时 

为什么中国专家不研究穿山甲？因为，早被吃没了 

为啥科幻小说里的打打打都要拼动力？ 

什么叫“外油内干”肤质啊？我到底该补水，还是该控油？ 

今晚吃汤圆or吃元宵？这根本不是正月十五的重点好吗！ 

令人们谈之色变的艾滋病，是从哪来的？ 

你不懂换挡：在你销魂动作的背后，变速箱都忙活些啥？ 

你以为侠盗猎车手是那么好当的吗？保命要紧！ 

你刺痛过的脸，干燥过的皮，归根结底都是这个功能有问题 

你说啥？寨卡疫情是转基因蚊子引起的？有没搞错啊！ 

别拿蚊子不当回事儿，它们的危害可不止让你痒得难受！ 

别整那些没用的！我只想要一支不作妖的防晒霜 

别看有些护肤品成分简单，确是敏感肌的大救星！ 好物推荐 

动力科技如何用1+1推动未来快进？ 

古代中国有没有入侵物种造成生态灾难的事情？ 

围绕“基因编辑婴儿”的荒诞剧，三百年前就已经上演了 

在沦落成“情人节”之前，古人的七夕比你们内涵多了 

夏天被蚊子吸血很烦，那...蚊子也会感染病毒吗？ 

嫦娥是谁？广寒在哪？中秋怎么来？你啥也不懂，就知道吃月饼！ 

广东查获1.6吨穿山甲鳞片：徒有一身坚甲，难抵人类贪婪 

思铂睿混动油耗有多省？我们把车子开上五环实际挑战 

愚人节：不能忘记的中国传统节日！ 

我要吹爆能卸妆的洗面奶，你们应该没有意见吧？ 

接受基因疗法后，这个失去皮肤的“蝴蝶男孩”活了下来 

提起“穿山甲”就想到“惨”？这9个萌点你必须得知道一下！ 

最后的盔犀鸟：这个活生生的物种，正一个个变成“工艺品” 

比起花样百出的月饼，我独爱月亮里温柔深沉的兔子 

治痘痘只会玩儿命上猛药？不同等级的痘痘要用不同治疗方法你知道吗 

泰国登革热疫情爆发，国内这些地方也有感染风险！预防方法都在这 

珍稀棱皮龟惨遭屠宰，海龟保育之路任重道远 

甭管小年是二十三还是二十四，你们送上

以上正文的展示省略了过多的内容。文档数量总共为48篇，分词后的文本以空格隔开词汇。接下来选取关键词，根据之前的计划，为了观察降维操作带来的影响，我们先不进行降维：

In [16]:
# n = -1意味着选取文档中出现的所有词汇
key_words_list = get_keywords(seg_articles_delstp, article_names, -1)
print('length: ', len(key_words_list), '\n')
print('key_words_list: ', key_words_list[:30], '......')

length:  12241 

key_words_list:  ['主力军', '嫌弃', '仙', '降级', '芙文', '下药', '手臂', '微盘', '行署', '惯例', '不起眼', '男同志', '独断', '哪来', '送交', '杂志', '应用领域', '行之君', '下为', '高庄', '防疟', '文化史', '黄海', '选项', '车祸', '未注明', '纸外', '人尽望', '牛马', '自古'] ......


我们得到一万两千多个词汇，其中含有一定数量的生僻词。

下面计算TF-IDF，我们对标题赋予0.005的权重，令最小文档频率和最大文档频率分别为0、48，即不进行任何约束。

In [17]:
title_weight = 0.005
min_df = 0
max_df = 48
tfidf, key_words_upd = cal_tf_idf(seg_articles, article_names, key_words_list, title_weight, min_df, max_df)
print('shape: ', tfidf.shape)

shape:  (48, 12241)


### 聚类

经过以上过程，我们已经获得了描述文档的特征，即TF-IDF权重。接着根据特征的值进行聚类，把特征相似的文档归为一类，将特征不相似的文档归在不同类别中。

在这里，我们选用最简单的聚类算法，即K-means聚类。K-means聚类算法的原理是：随机选取类的中心点，根据样本点到中心点的距离将样本归类，然后定义同一类中所有样本点的重心为新的中心点，如此迭代直到中心点位置不发生显著的改变。这一算法是根据欧式距离计算样本点与样本点之间的距离，但我们的特征位于高维空间，并且特征向量的长度取决于文档长度等因素，为了避免这些因素的影响，应当用向量之间的夹角来描述文档相似度而并非直接计算欧氏距离。同时注意到两个向量的夹角和将它们归一化后再计算的空间距离存在一一对应的关系，换句话说，如果将所有的特征向量映射到单位球上，计算夹角与空间距离事实上是同一回事。所以在使用K-means算法之前，首先要对权重进行归一化。

In [18]:
%%time
# 归一化权重
tf_idf_norm = normalize(tfidf)

# 选取合适的类别数量，这里经过尝试，我们选择10类
n_clusters = 10

# K-means
#为了得到相对可靠与稳定的结果，选择一个较大的n_init，这里用不同初始值重复2000次K-means算法，
labels = KMeans(n_clusters=n_clusters, n_init=2000).fit_predict(tf_idf_norm)

Wall time: 1min 27s


In [19]:
article_label = pd.Series(article_names, index=labels)
for label in range(n_clusters):   
    print('class ', label, ': ')
    if isinstance(article_label[label], str):
        print(article_label[label])
    else:
        for name in article_label[label]:
            print(name)
    print()

class  0 : 
你说啥？寨卡疫情是转基因蚊子引起的？有没搞错啊！
别拿蚊子不当回事儿，它们的危害可不止让你痒得难受！
夏天被蚊子吸血很烦，那...蚊子也会感染病毒吗？
泰国登革热疫情爆发，国内这些地方也有感染风险！预防方法都在这
蚊子：我活不了，你也别想好过

class  1 : 
嫦娥是谁？广寒在哪？中秋怎么来？你啥也不懂，就知道吃月饼！
比起花样百出的月饼，我独爱月亮里温柔深沉的兔子

class  2 : 
7位科学家评基因编辑婴儿：不是对新知的探索，也没有医疗意义
“基因编辑婴儿”出现后，设计一个“完美婴儿”离我们还有多远？ 热点
“艾滋病免疫婴儿”诞生：基因编辑技术，为何成为争议的漩涡？丨热点
围绕“基因编辑婴儿”的荒诞剧，三百年前就已经上演了
接受基因疗法后，这个失去皮肤的“蝴蝶男孩”活了下来
贺建奎自称“借鉴了”权威伦理指南，但他一条也没遵守

class  3 : 
什么叫“外油内干”肤质啊？我到底该补水，还是该控油？
你刺痛过的脸，干燥过的皮，归根结底都是这个功能有问题
别整那些没用的！我只想要一支不作妖的防晒霜
别看有些护肤品成分简单，确是敏感肌的大救星！ 好物推荐
我要吹爆能卸妆的洗面奶，你们应该没有意见吧？
治痘痘只会玩儿命上猛药？不同等级的痘痘要用不同治疗方法你知道吗
白色情人节？别慌，回赠ta一片“海枯石烂”

class  4 : 
为什么中国专家不研究穿山甲？因为，早被吃没了
广东查获1.6吨穿山甲鳞片：徒有一身坚甲，难抵人类贪婪
提起“穿山甲”就想到“惨”？这9个萌点你必须得知道一下！
穿山甲：中国人的无知贪婪，让它们离灭绝一步之遥

class  5 : 
古代中国有没有入侵物种造成生态灾难的事情？
最后的盔犀鸟：这个活生生的物种，正一个个变成“工艺品”
珍稀棱皮龟惨遭屠宰，海龟保育之路任重道远
面对蝗灾，唐太宗选择了生！吞！蚂！蚱！

class  6 : 
今晚吃汤圆or吃元宵？这根本不是正月十五的重点好吗！
在沦落成“情人节”之前，古人的七夕比你们内涵多了
愚人节：不能忘记的中国传统节日！
甭管小年是二十三还是二十四，你们送上天的灶王都是个渣男
腊八：节日中的落难贵族？那是两大节日的合♂体！
重阳节是“灾日”吗？登高吃菊花就能长寿？茱萸又是谁？
鸡年将至，让我们来扒一扒鸡鸡的黑历史

class  7 : 
薯片、

我们发现，在同一类中存在的大多都是具有相似主题的文章，比如有关“基因编辑”的文章被归为同一类，同时还能发现其他主题，例如“寨卡病毒”、“汽车科技”、“传统节日”等。

同时我们发现一些文章被归到与期望不符的类别，例如对于类别9的“鹡鹚鹬鵙鹩鹛鹮鸺鹠鹪鹩䴙䴘——这都是些什么鸟字？”这篇文章，我们更希望将其归为类别5，因为这篇文章明显同类别中其他文章的“病毒”主题无关。

另外，类别与类别之间存在不均匀性，比如类别7的两篇文章高度相似，而类别5的文章虽然都涉及物种的问题，但具有不同的主题。

总体而言，本次聚类结果比较令人满意，虽然特征空间有一万多维（没有进行任何降维操作），但是K-means算法仍产生了大致准确的归类。接下来，我们尝试对特征空间进行降维。

### 数据预处理 & 聚类 —— 降维后的结果

#### 特征选取

对于每篇文档，选取频率前30的词汇作为关键词。可以看到关键词的数量明显下降——只有一千多个。

In [20]:
key_words_list = get_keywords(seg_articles_delstp, article_names, 30)
print('length: ', len(key_words_list), '\n')
print('key_words_list: ', key_words_list[:30], '......')

length:  1086 

key_words_list:  ['治', '蓝', '前', '祛痘', '汽车', '体细胞', '深沉', '干净', '吸', '进化', '配子体', '学', '哪来', '伦理', '蟾蜍', '组合', '龟', '农户', '动力', '文献', '行星', '翻', '记载', '找到', '我国', '化石', '事', '有害', '野生', '组委会'] ......


去除在所有文档中出现次数过小或过大的词汇，规定下限为2，上限为30。关键词的数量又一次降低。

经过降维操作，特征数量从之前的一万多个下降为七百多个。

In [21]:
title_weight = 0.005
min_df = 2
max_df = 30
tfidf, key_words_upd = cal_tf_idf(seg_articles, article_names, key_words_list, title_weight, min_df, max_df)
print('shape: ', tfidf.shape)

shape:  (48, 719)


In [22]:
%%time
tf_idf_norm = normalize(tfidf)
n_clusters = 10
labels = KMeans(n_clusters=n_clusters, n_init=2000).fit_predict(tf_idf_norm)

Wall time: 7.62 s


In [23]:
article_label = pd.Series(article_names, index=labels)
for label in range(n_clusters):   
    print('class ', label, ': ')
    if isinstance(article_label[label], str):
        print(article_label[label])
    else:
        for name in article_label[label]:
            print(name)
    print()

class  0 : 
今晚吃汤圆or吃元宵？这根本不是正月十五的重点好吗！
在沦落成“情人节”之前，古人的七夕比你们内涵多了
愚人节：不能忘记的中国传统节日！
甭管小年是二十三还是二十四，你们送上天的灶王都是个渣男
腊八：节日中的落难贵族？那是两大节日的合♂体！
重阳节是“灾日”吗？登高吃菊花就能长寿？茱萸又是谁？
鸡年将至，让我们来扒一扒鸡鸡的黑历史

class  1 : 
“M型病毒”这个名字哪儿来的？病毒命名有什么规律？
世界艾滋病日：阻断HIV感染的黄金24小时
令人们谈之色变的艾滋病，是从哪来的？
病毒竟然也有免疫系统！为什么？因为怕被其他病毒感染

class  2 : 
什么叫“外油内干”肤质啊？我到底该补水，还是该控油？
你刺痛过的脸，干燥过的皮，归根结底都是这个功能有问题
别整那些没用的！我只想要一支不作妖的防晒霜
别看有些护肤品成分简单，确是敏感肌的大救星！ 好物推荐
我要吹爆能卸妆的洗面奶，你们应该没有意见吧？
接受基因疗法后，这个失去皮肤的“蝴蝶男孩”活了下来
治痘痘只会玩儿命上猛药？不同等级的痘痘要用不同治疗方法你知道吗

class  3 : 
薯片、蛋卷和爆米花：人为什么喜欢酥脆的食物？
薯片炸鸡爆米花，你为啥喜欢嘎嘣脆的食物？

class  4 : 
你说啥？寨卡疫情是转基因蚊子引起的？有没搞错啊！
别拿蚊子不当回事儿，它们的危害可不止让你痒得难受！
夏天被蚊子吸血很烦，那...蚊子也会感染病毒吗？
泰国登革热疫情爆发，国内这些地方也有感染风险！预防方法都在这
蚊子：我活不了，你也别想好过

class  5 : 
一个灵敏的屁股有什么用
为啥科幻小说里的打打打都要拼动力？
你不懂换挡：在你销魂动作的背后，变速箱都忙活些啥？
你以为侠盗猎车手是那么好当的吗？保命要紧！
动力科技如何用1+1推动未来快进？
思铂睿混动油耗有多省？我们把车子开上五环实际挑战

class  6 : 
嫦娥是谁？广寒在哪？中秋怎么来？你啥也不懂，就知道吃月饼！
比起花样百出的月饼，我独爱月亮里温柔深沉的兔子

class  7 : 
7位科学家评基因编辑婴儿：不是对新知的探索，也没有医疗意义
“基因编辑婴儿”出现后，设计一个“完美婴儿”离我们还有多远？ 热点
“艾滋病免疫婴儿”诞生：基因编辑技术，为何成为争议的漩涡？丨热点
围绕“基因编辑婴儿”的荒诞剧

同时，维度的减少也减少了K-means计算的时间，降维后的聚类结果仍然较为合理。说明选取的特征还是保留了主要用来区别文章的信息，在减少维度的同时也保证了聚类的准确性。

#### 转化为相似度矩阵

在上一个小节，我们通过特征选取将关键词数量减少至将近5%，并同样得到了较为准确的聚类结果，但是此时对于48篇文章的小样本而言，特征空间还是有719维，远远大于样本数量。这说明，向量空间依然过于稀疏，我们希望进一步的处理，使得维度大小与样本数量接近或更小。

我们尝试将描写特征的权重改为相似度，也就是每篇文档的特征及它与其他文档的相似度，这时我们获得48 * 48的相似度矩阵，每一行描述一篇文档。这一做法直接使空间维度将至与文档数量相同的大小，防止“维度灾难”，于此同时，用相似度代表特征虽然丢失了文档本身关键词的信息，但是保留了文档之间的差别信息，仍然可以用来区分不同类型的文档。

In [24]:
# 先定义计算相似度的函数，注意计算的是余弦相似度
def cal_similarity(mtx):
    n_articles = mtx.shape[0]
    sim = np.zeros([n_articles, n_articles])
    mtx_length = ((mtx * mtx).sum(1)) ** (0.5)
    
    for i in range(n_articles):
        for j in range(i, n_articles):
            sim[i, j] = ((mtx[i] * mtx[j]).sum()) / (mtx_length[i] * mtx_length[j])
            sim[j, i] = sim[i, j]
    
    return sim

In [25]:
sim = cal_similarity(tfidf)
n_clusters = 10
labels = KMeans(n_clusters=n_clusters, n_init=2000).fit_predict(sim)

article_label = pd.Series(article_names, index=labels)
for label in range(n_clusters):   
    print('class ', label, ': ')
    if isinstance(article_label[label], str):
        print(article_label[label])
    else:
        for name in article_label[label]:
            print(name)
    print()

class  0 : 
“M型病毒”这个名字哪儿来的？病毒命名有什么规律？
世界艾滋病日：阻断HIV感染的黄金24小时
令人们谈之色变的艾滋病，是从哪来的？
病毒竟然也有免疫系统！为什么？因为怕被其他病毒感染

class  1 : 
古代中国有没有入侵物种造成生态灾难的事情？
最后的盔犀鸟：这个活生生的物种，正一个个变成“工艺品”
泰国登革热疫情爆发，国内这些地方也有感染风险！预防方法都在这
珍稀棱皮龟惨遭屠宰，海龟保育之路任重道远
白色情人节？别慌，回赠ta一片“海枯石烂”
面对蝗灾，唐太宗选择了生！吞！蚂！蚱！
鸡年将至，让我们来扒一扒鸡鸡的黑历史
鹡鹚鹬鵙鹩鹛鹮鸺鹠鹪鹩䴙䴘——这都是些什么鸟字？

class  2 : 
你说啥？寨卡疫情是转基因蚊子引起的？有没搞错啊！
别拿蚊子不当回事儿，它们的危害可不止让你痒得难受！
夏天被蚊子吸血很烦，那...蚊子也会感染病毒吗？
蚊子：我活不了，你也别想好过

class  3 : 
为什么中国专家不研究穿山甲？因为，早被吃没了
广东查获1.6吨穿山甲鳞片：徒有一身坚甲，难抵人类贪婪
提起“穿山甲”就想到“惨”？这9个萌点你必须得知道一下！
穿山甲：中国人的无知贪婪，让它们离灭绝一步之遥

class  4 : 
一个灵敏的屁股有什么用
为啥科幻小说里的打打打都要拼动力？
你不懂换挡：在你销魂动作的背后，变速箱都忙活些啥？
你以为侠盗猎车手是那么好当的吗？保命要紧！
动力科技如何用1+1推动未来快进？
思铂睿混动油耗有多省？我们把车子开上五环实际挑战

class  5 : 
今晚吃汤圆or吃元宵？这根本不是正月十五的重点好吗！
在沦落成“情人节”之前，古人的七夕比你们内涵多了
愚人节：不能忘记的中国传统节日！
甭管小年是二十三还是二十四，你们送上天的灶王都是个渣男
腊八：节日中的落难贵族？那是两大节日的合♂体！
重阳节是“灾日”吗？登高吃菊花就能长寿？茱萸又是谁？

class  6 : 
什么叫“外油内干”肤质啊？我到底该补水，还是该控油？
你刺痛过的脸，干燥过的皮，归根结底都是这个功能有问题
别整那些没用的！我只想要一支不作妖的防晒霜
别看有些护肤品成分简单，确是敏感肌的大救星！ 好物推荐
我要吹爆能卸妆的洗面奶，你们应该没有意见吧？
接受基因疗法后，这个失去皮肤的“蝴蝶男孩”活了下来
治痘痘只会玩儿命上猛药？

用相似度描述特征的方法得到与之前类似的聚类结果，说明这一方法具有一定合理性。

#### PCA降维

当样本数量很大时，转换为相似度的特征空间依然有着很高的维度，这时之前的方法均不能满足要求。因此，我们尝试采用PCA。

PCA基于高维空间数据分布稀疏，所以必然导致一些维度是多余的思想，将数据投影到低维空间，并尽可能保持方差。下面的算法中，我们保留95%的方差，可以看到，新的维度为38维，小于样本数量。（事实上，PCA降维必然使得维度小于样本数量，因为n个样本点最多确定n-1维的空间。）

In [26]:
pca = PCA(n_components=0.95)
# 注意这里是用标准化之后的TF-IDF矩阵
tf_idf_pca = pca.fit_transform(tf_idf_norm)
cumsum = np.cumsum(pca.explained_variance_ratio_)
print(pca.n_components_)

38


In [28]:
n_clusters = 10
labels = KMeans(n_clusters=n_clusters, n_init=2000).fit_predict(tf_idf_pca)

article_label = pd.Series(article_names, index=labels)
for label in range(n_clusters):   
    print('class ', label, ': ')
    if isinstance(article_label[label], str):
        print(article_label[label])
    else:
        for name in article_label[label]:
            print(name)
    print()

class  0 : 
嫦娥是谁？广寒在哪？中秋怎么来？你啥也不懂，就知道吃月饼！
比起花样百出的月饼，我独爱月亮里温柔深沉的兔子

class  1 : 
为什么中国专家不研究穿山甲？因为，早被吃没了
广东查获1.6吨穿山甲鳞片：徒有一身坚甲，难抵人类贪婪
提起“穿山甲”就想到“惨”？这9个萌点你必须得知道一下！
穿山甲：中国人的无知贪婪，让它们离灭绝一步之遥

class  2 : 
“M型病毒”这个名字哪儿来的？病毒命名有什么规律？
世界艾滋病日：阻断HIV感染的黄金24小时
令人们谈之色变的艾滋病，是从哪来的？
你不懂换挡：在你销魂动作的背后，变速箱都忙活些啥？
病毒竟然也有免疫系统！为什么？因为怕被其他病毒感染
白色情人节？别慌，回赠ta一片“海枯石烂”
鸡年将至，让我们来扒一扒鸡鸡的黑历史

class  3 : 
你说啥？寨卡疫情是转基因蚊子引起的？有没搞错啊！
别拿蚊子不当回事儿，它们的危害可不止让你痒得难受！
夏天被蚊子吸血很烦，那...蚊子也会感染病毒吗？
泰国登革热疫情爆发，国内这些地方也有感染风险！预防方法都在这
蚊子：我活不了，你也别想好过

class  4 : 
什么叫“外油内干”肤质啊？我到底该补水，还是该控油？
你刺痛过的脸，干燥过的皮，归根结底都是这个功能有问题
别整那些没用的！我只想要一支不作妖的防晒霜
别看有些护肤品成分简单，确是敏感肌的大救星！ 好物推荐
我要吹爆能卸妆的洗面奶，你们应该没有意见吧？
接受基因疗法后，这个失去皮肤的“蝴蝶男孩”活了下来
治痘痘只会玩儿命上猛药？不同等级的痘痘要用不同治疗方法你知道吗

class  5 : 
一个灵敏的屁股有什么用
为啥科幻小说里的打打打都要拼动力？
你以为侠盗猎车手是那么好当的吗？保命要紧！
动力科技如何用1+1推动未来快进？
思铂睿混动油耗有多省？我们把车子开上五环实际挑战

class  6 : 
今晚吃汤圆or吃元宵？这根本不是正月十五的重点好吗！
在沦落成“情人节”之前，古人的七夕比你们内涵多了
愚人节：不能忘记的中国传统节日！
甭管小年是二十三还是二十四，你们送上天的灶王都是个渣男
腊八：节日中的落难贵族？那是两大节日的合♂体！
重阳节是“灾日”吗？登高吃菊花就能长寿？茱萸又是谁？

class  7 : 
古代中国有没有入侵物种造成生态灾难的事情？
最后的盔犀鸟：

采用PCA降维的结果依然较为合理。

### 小结

以上我们采用了四套方法，下面将它们列举出来：

1. 选用所有文档的所有非停用词汇作为关键词。维度大小：12241。
2. 选用每篇文档频率前30的词汇作为关键词，并去除DF值过小或过大的词汇。维度大小：719。
3. 在 2 的基础上计算文档与文档的相似度，用相似度代替权重。维度大小：48。
4. 在 2 的基础上采用PCA降维。维度大小：38

结果显示，三种降维处理的方式均能产生合理的聚类结果。而降维效果则是特征选取+PCA的方式最优。

## 大样本处理

接下来进行大样本的处理，对于样本数量为几百几千的情况，由于关键词数量随着文档数量增加而增加，特征空间也会有较高的维度。根据我们之前在小样本上实验的结果，为了尽可能地减少空间，采用特征选取+PCA的降维方法。

### 数据导入

我们从已知的历史文章网址中获取html页面，并从页面中提取所需的文本数据。

In [29]:
def extract_content(url):
    
    # 根据url地址提取正文内容
    
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    content = soup.select('#js_content')
    
    article_str = ''
    for i in range(len(content)):
        article_str += content[i].text
        
    return article_str

In [ ]:
# 有关微信公众号文章信息的内容均储存于"guokr_articles.csv"文件中
# 读取文件
f = open('guokr_articles.csv', 'rb')
df = pd.read_csv(f)
f.close()
# 选择最近的1000篇文章作为数据
articles_df = df.loc[9180:, ['时间', '标题', '文章链接']]
n_articles = articles_df.shape[0]
articles_df.index = range(n_articles)

In [30]:
# 定义文档数据存放路径
articles_path = r'F:\\guokr_articles\\'

### 数据预处理 & 数据分析

以下内容与小样本上的处理流程相同。

In [31]:
# 存放文章标题
article_names_full = []
# 存放原始文本内容
article_texts_full = []
# 存放分词后的文本内容
seg_articles_full = []
# 存放分词、去停用词后的文本内容
seg_articles_delstp_full = []

# 一一读取目录中的文档
for fname in os.listdir(articles_path):
    article = open(os.path.join(articles_path, fname),'r',encoding='utf-8').read()
    
    article_name = fname.replace('.txt', '')
    article_names_full.append(article_name)
    
    article_text = article.replace('\n', '')
    article_texts_full.append(article_text)
    # 分词
    article_seg_list = jieba.lcut(article_text)
    
    seg_text = ''
    seg_text_delstp = ''
    
    for word in article_seg_list:
        if is_chinese(word):
            seg_text += word + ' '
            if word not in stopwords_list:
                seg_text_delstp += word + ' '
                
    seg_articles_full.append(seg_text)
    seg_articles_delstp_full.append(seg_text_delstp)

In [32]:
n_articles_full = len(article_names_full)
print('n_articles: ', n_articles_full, '\n')
print('article_names: ', article_names_full[0], '\n')
print('article_texts: ', article_texts_full[0][:200], '......', '\n')
print('seg_articles: ', seg_articles_full[0][:200], '......', '\n')
print('seg_articles_delstp: ', seg_articles_delstp_full[0][:200], '......', '\n')

n_articles:  1000 

article_names:  &quot;毒书皮&quot;卷土重来，强致癌、或致儿童性早熟？ 

article_texts:  又到了一年一度的开学季，新一轮的“包书皮大战”也拉开了序幕，正所谓“举头望明月，低头包书皮”，“洛阳亲友如相问，我在家中包书皮”。近年来，塑料包书皮因其样式繁多、图案鲜艳，兼具防水、耐磨、防污染的功能，备受广大中小学生青睐。但最近网上有消息称，这些美丽夺目的包书皮很可能含有致癌性的多环芳烃和导致性早熟的邻苯二甲酸酯…… 2015 年，“毒书皮事件”火遍网络。物理系毕业的魏先生盯着女儿的包书皮，怎么 ...... 

seg_articles:  又 到 了 一年一度 的 开学 季 新一轮 的 包书皮 大战 也 拉开 了 序幕 正 所谓 举头 望明月 低头 包书皮 洛阳 亲友 如相问 我 在 家中 包书皮 近年来 塑料 包书皮 因 其 样式 繁多 图案 鲜艳 兼具 防水 耐磨 防污染 的 功能 备受 广大 中小学生 青睐 但 最近 网上 有 消息 称 这些 美丽 夺目 的 包书皮 很 可能 含有 致癌性 的 多环 芳烃 和 导致 性早熟 的 ...... 

seg_articles_delstp:  一年一度 开学 季 新一轮 包书皮 大战 拉开 序幕 正 举头 望明月 低头 包书皮 洛阳 亲友 如相问 家中 包书皮 塑料 包书皮 样式 繁多 图案 鲜艳 兼具 防水 耐磨 防污染 功能 备受 中小学生 青睐 网上 消息 称 美丽 夺目 包书皮 含有 致癌性 多环 芳烃 导致 性早熟 邻苯二甲酸 酯 年 毒 书皮 事件 火遍 网络 物理系 毕业 魏先生 盯 女儿 包书皮 闻 一股 刺鼻 臭味儿  ...... 



In [33]:
key_words_list_full = get_keywords(seg_articles_delstp_full, article_names_full, 30)
print('length: ', len(key_words_list_full), '\n')
print('key_words_list: ', key_words_list_full[:30], '......')

title_weight = 0.005
min_df = 2
max_df = 800
tfidf_full, key_words_upd_full = cal_tf_idf(seg_articles_full, article_names_full, key_words_list_full, title_weight, min_df, max_df)
print('shape: ', tfidf_full.shape, '\n')

tf_idf_full_norm = normalize(tfidf_full)
pca = PCA(n_components=0.95)
# 注意这里是用标准化之后的TF-IDF矩阵
tf_idf_full_pca = pca.fit_transform(tf_idf_full_norm)
print('pca components: ', pca.n_components_)

n_clusters = 200
labels_full = KMeans(n_clusters=n_clusters, n_init=200).fit_predict(tf_idf_full_pca)

article_label_full = pd.Series(article_names_full, index=labels_full)
for label in range(n_clusters):
    print('class ', label, ': ')
    if isinstance(article_label_full[label], str):
        print(article_label_full[label])
    else:
        for name in article_label_full[label]:
            print(name)
    print()

length:  11912 

key_words_list:  ['引脚', '仙', '年份', '治死', '视觉', '扫描', '心绞痛', '甘纳许', '扭蛋里', '手臂', '屯', '香螺', '颈椎病', '不起眼', '纸币', '熟醉蟹', '水晶球', '哪来', '杂志', '课程', '生成', '空间站', '吃糖', '选项', '车祸', '手气', '一毛钱', '姬', '进化论', '说唱'] ......
shape:  (1000, 9666) 

pca components:  843
class  0 : 
休斯顿，我们的空间站被砸漏了！用胶带粘粘能行吗  迈向太空
俄罗斯最可靠的火箭出故障了！宇航员险些丧生
再见，大钟！我国新一代载人飞船重磅亮相，目标直指载人登月

class  1 : 
10次雨果奖、7次星云奖、4次轨迹奖。顶尖科幻大师解构“时间”  好物推荐
刘慈欣获克拉克奖，这个奖可能⽐任何奖都重得多
韩松：科幻预言超级消费时代丨有意思博物馆

class  2 : 
下周一就是教师节？别急，礼物已经挑好了！ 好物推荐
丝巾怎么系，才能美美哒？ 好物推荐
在大阪，享受美食一定要去这个地方  好物推荐
江湖救急！距离七夕只剩5天，现在挑礼物还来得及
身价N个亿的大佬们，退休后坐拥金山会干啥？

class  3 : 
一份连环杀手的占星解读，让94%的受访者看到了自己
为了证明太阳小时候很熊，我们找遍了它的“三姑六婆” 迈向太空
太阳系或有第九行星，比地球还大？新天体的发现再添线索  迈向太空
帕克号：种不成太阳，那还可以摸它呀！ 迈向太空
飞出太阳系！旅行者探测器将飞往何处？

class  4 : 
“妈，求你别再转发这种文章了”，中老年人为何成为网络谣言重灾区？
你家宝宝为什么总是吃手手？可不是为了卖萌
健身网红挺着大肚子运动，婆婆却叫我老实躺着，我究竟该听谁的？ 过日子
喘不上气，还拿纸袋罩住脸？可能真的有用！
在网上，没人在意你120岁
我国有2.1亿人的健康，被”沉默的杀手“控制
有一种痛，让人彻底告别啤酒和海鲜，它的名字叫痛风  过日子
有的人活着，但他的颈椎已经快死了……
爱运动的老人家们，健身不能光顾着动作酷炫啊！丨过日子
老人可能不是真耳背，而是这个小问题造成的
脊柱里装上小电脑，或

晚上吃姜赛砒霜、隔夜水不能喝？“生活小窍门”的真相在这里  AI夜话
用“祛痘神药”的这段日子，没想到这么难熬
节后上班怎么都爆痘了？有一种急痘攻心叫压力很大

class  151 : 
为什么你吃不到真阳澄湖大闸蟹？它们没被捞起就已经被卖掉啦！
怎样买买买才能又走心还不伤肾？
距离中秋只剩4天了，你是不是忘了些什么？ 好物推荐
阳澄湖终于开湖了，如何才能避免买到“洗澡蟹”？ 好物推荐

class  152 : 
自从知道红领犯罪，我开始感激每一位不杀我的同事
谢同事不杀之恩……工作场所排第三的死因，竟然是谋杀！

class  153 : 
6.97吨碳九泄露，危害会有多大？丨热点
99%的人都不知道，易拉罐里还藏着一个透明的包装
“哎呀，明明咱俩干的活一样，怎么我拿的钱比你多这么多啊？”
“喂，专业名词，你是魔鬼吗？”
“国民零食”九制话梅，是哪九制？
“抽动秽语综合征”都被拿来骂人了，能别这么叫我么？
“表情移植神器”上线，我家爱豆的面瘫演技是不是有救了！
《风味人间》里没说完的馕，这次给你说得明明白白
一只鹿说：别吓唬我，信不信我“汪”地一声叫出来？！
一滴水就能做的显微镜，快点来试试
不切不尝不咬开，商家是怎么测水果甜度的？
不是我不想穿越到过去，是我舍不得空调和暖气
世界上最坚硬的食物，我打赌你一定吃过！
世界上最小的青蛙，能在你指甲盖上跳舞！
为了避免物种大灭绝，一半地球都该划成保护区，问题是划哪半？
为什么中国99.8%的鸟都是由外国人发现和命名的？
为什么总用五角星表示星星？人家明明是球体啊！
为什么有人放假第一天就写完了作业，而你开学前夜还在赶
为什么汤圆会在沸水中翻滚，煮饺子却不会？
为什么硬核游戏《太吾绘卷》独宠斗蛐蛐？因为真的很好玩啊
为什么科学家、医生、教授要穿白大褂？
为什么蟑螂喜欢爬进人耳朵里安家？ 过日子
为啥火车票和身份证一样大？又被检票员一块儿剪了！
为啥语文永远考不了高分老师们这么说……  好课推荐
亚马逊无人超市开业了，我在里面“偷”了样东西
人类最早具象画被发现啦，你猜猜画的是啥动物
人类这么热衷探索宇宙，为什么只有两艘探测器去过水星？  迈向太空
人肉的营养价值有多高？人的哪个内脏最有饱腹感？
什么样的空气质量，让你的鼻子分分钟要罢工？ 有奖问卷调查
什么汤能喝出风起云涌的味道？一定是味噌汤！
今天是世界预防自杀日：

算法将部分相似主题的文章成功分到了同一类别中，比如类别0有关航空航天，类别2有关科幻等等，同时，也有一些文章由于分布在稀疏区域等原因，被错误地与其他文章归为同一类，比如类别10。对于这种情况，我们可以专门提取这一类别的文章，再次进行聚类。

## 总结与分析

本次实验中，我们提取微信公众号的文章数据，采用K-means聚类算法。遇到的主要问题则是特征空间维度过高，使数据分布过于稀疏。为了解决“维度灾难”的问题，我们用不同的方法对特征进行降维。实验证明，降维后的特征仍能反映文章的主要信息，不会显著降低分类的合理性，同时通过减少数据量加快了算法的速度。

当我们将相同的算法作用于文章量为1000的较大样本，发现分类后的结果存在许多噪声与错误，这由多种原因造成：

1. 特征选取、PCA降维均有可能筛去能反映文章区别的关键词汇，从而减少有效信息，影响算法准确度。
2. K-means算法本身存在局限性。在聚类时需要规定类的数量多少，但事实上我们对应该分多少类并没有概念。
3. 样本集本身包含许多难以处理的样本。比如：文章长度过小，主题偏僻导致难以找到相似的文档。
4. 数据分布无规律、范围广。公众号的文章往往涉及生活中的方方面面，且不存在规律，不同类型的文章数量存在不平衡的问题。主题涵盖的范围甚广，使得样本原本的分布就比较稀疏。
5. 文本内容复杂多变，语言具有复杂性，一词多义、新词的出现降低了关键词的描述的准确性和全面性。
6. 以关键词作为特征的局限性。关键词虽然反映文档的一部分信息，但是忽略了语义层面的信息。首先，关键词之间可能存在同义、近义等关系，在模型中并未考虑词与词之间的关系，不同的词均统一代表不同维度。其次，文章的主题应通过对整篇文章的总体理解，但是关键词则只考虑单一、分裂的词，忽略了文章的整体结构与含义。

为了解决这些问题，我认为日后应当重视语义层面的信息，以下提出几点猜想：

1. 考虑关键词之间的语义关系、语义相似度，对同义词进行合并。
2. 引入知识库，从关键词入手分析归纳文章所涉及的主题或领域。
3. 对标题进行语义理解将有利于利用最少的数据获取大量信息，但是如何让算法理解语言文字传达的信息成为问题。

## 参考文献

1. https://github.com/fxsjy/jieba
2. https://cuiqingcai.com/1319.html
3. https://blog.csdn.net/shijiebei2009/article/details/39696571
4. http://brandonrose.org/clustering
5. http://brandonrose.org/top100
6. https://scikit-learn.org/stable/index.html